In [41]:
import pypostmaster
import getpass
from queryrunner_client import *
from queryrunner_client import QueryRunnerException
import numpy as np
import pandas as pd
from pandas.io import gbq
import pytz
from datetime import datetime, timedelta, tzinfo
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
import docx
from docx import Document
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Inches
import jinja2
import math
import yaml
import os
import signal

import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import uuid
import json
from google.cloud import bigquery
import base64
import nbformat
from IPython import get_ipython
import shutil
import subprocess
import random
from croniter import croniter


ipython = get_ipython()

In [42]:
from LP_helper import ymlParser 
from LP_helper import validateDriveAccess
from LP_helper import runBQuery
from LP_helper import getNextID
from LP_helper import getDateFromShortName
from LP_helper import getDqrRequest
from LP_helper import isDqrRequestAvailable
from LP_helper import executeNotebookCell
from LP_helper import postDqrCompleted
from LP_helper import sendDqrMail
from LP_helper import postDqrScriptFail
from LP_helper import CreateDqrLogger
from LP_helper import resetDqrLongRunningTask
from LP_helper import maxDqrTaskChecker
from LP_helper import resetDqrForTesting
from LP_helper import postDqrAutoLog
from LP_helper import Fail_mail


from LP_helper import uploadFileToDrive
from LP_helper import getQueryUUID
from LP_helper import getEncodedImg

from LP_helper import getDpvQueryReportID
from LP_helper import getDataPushRequest
from LP_helper import triggerDataPush
from LP_helper import getDataPushReportRequest
from LP_helper import postDpvCompleted
from LP_helper import sendDpvMail

from LP_helper import getHistoricalScheduler
from LP_helper import isScheduled
from LP_helper import nextRunin
from LP_helper import SchedulesToPrecheck
from LP_helper import getThisWeekStartEnd
from LP_helper import getSchedulerForPreCheck
from LP_helper import postReportRequest
from LP_helper import postSchedulerToDqrRequest

from LP_helper import getData

# Schedule to Pre-Check

In [43]:
historicalScheduler = getHistoricalScheduler()
df = pd.DataFrame(historicalScheduler)
df

,scheduler_id,created_at,updated_at,last_executed_at,is_active,dqr_name,run_on,run_condition,run_status,email_address,query_run_condition,getRunId_querybuilder,cron_minute,cron_hour,cron_day,cron_month,cron_weekday,dashboard,parameters,cronEcpresssion
0,114,2025-01-27 10:37:04,2025-01-30 05:32:13,2025-01-30 11:02:12,1,Manual_Handled_NPI_method2_NPI_premier,mrajor,Recurring;Until Complete,None,mrajor@ext.uber.com,None,None,0,11,*,*,*,Marketplace Ops,{},0 11 * * *


In [44]:
dashboard_dict = {} 
for schedule in historicalScheduler:
    status = isScheduled(schedule['last_executed_at'], schedule['cronEcpresssion'])
    
    if status: 
        print(schedule)
        SchedulesToPrecheck(schedule)
        # dashboard_list.append(schedule['dashboard'])
        dashboard_dict[schedule['scheduler_id']] = schedule['dashboard']
    print(status, schedule['dqr_name'],schedule['dashboard'])
    dashboard = schedule['dashboard']
    # dashboard_list.append(dashboard)
    nextRunin(schedule['cronEcpresssion'])
print(dashboard_dict)
    

False Manual_Handled_NPI_method2_NPI_premier Marketplace Ops
0:57:01.762118
{}


# Pre-Check to DQR request

In [45]:
onPreCheck = getSchedulerForPreCheck()
onPreCheck



            SELECT * FROM launchpad_schedular_on_pre_check_dev
            WHERE run_status != 'Completed'
            AND (
                past_checked_at IS NULL OR 
                past_checked_at < '2025-01-31 09:43:04'
            )
        
   pre_check_id  scheduler_id          created_at          updated_at  \
0           234           119 2025-01-31 04:32:13 2025-01-31 04:32:13   

  getRunId_querybuilder run_status runIds past_checked_at parameters  
0                  None                              None         {}  


[{'pre_check_id': 234,
  'scheduler_id': 119,
  'created_at': Timestamp('2025-01-31 04:32:13'),
  'updated_at': Timestamp('2025-01-31 04:32:13'),
  'getRunId_querybuilder': None,
  'run_status': '',
  'runIds': '',
  'past_checked_at': None,
  'parameters': '{}'}]

In [39]:
# from pandas import Timestamp
# onPreCheck = [{'pre_check_id': 220,
#   'scheduler_id': 120,
#   'created_at': Timestamp('2025-01-30 17:26:06'),
#   'updated_at': Timestamp('2025-01-30 17:26:06'),
#   'getRunId_querybuilder': None,
#   'run_status': '',
#   'runIds': '',
#   'past_checked_at': None,
#   'parameters': '{"run_id": ""}'},
#  {'pre_check_id': 221,
#   'scheduler_id': 121,
#   'created_at': Timestamp('2025-01-30 17:26:07'),
#   'updated_at': Timestamp('2025-01-30 17:26:07'),
#   'getRunId_querybuilder': None,
#   'run_status': '',
#   'runIds': '',
#   'past_checked_at': None,
#   'parameters': '{"text1": "launchpad", "text2": "launchpad parameter", "c_code": "Hh9w3g6hR", "c_name": "onEk0k6nx"}'}]
# onPreCheck = [{'pre_check_id': 184,
#   'scheduler_id': 106,
#   'created_at': Timestamp('2024-12-16 04:57:03'),
#   'updated_at': Timestamp('2024-12-16 04:57:03'),
#   'getRunId_querybuilder': None,
#   'run_status': '',
#   'runIds': '',
#   'past_checked_at': None,
#   'parameters': '{"text1": "launchpad", "text2": "launchpad parameter", "c_code": "Hh9w3g6hR", "c_name": "onEk0k6nx"}'}]

In [47]:
list(map(lambda x: postSchedulerToDqrRequest(x, dashboard), onPreCheck))
# list(map(lambda x: postSchedulerToDqrRequest(x, dashboard_dict.get(x['scheduler_id'], 'default_dashboard')), onPreCheck))

param keys: []
status: Completed


json param:{}
Query executed successfully.
{}
Query executed successfully.
MySQL connection is closed


[True]

In [8]:
# list(map(postSchedulerToDqrRequest, onPreCheck,dashboard))

In [ ]:
parameters = {'query_id': 'uCidl3haP'}
param_type = {'query_id': 'qb_report'}

In [ ]:
param_type = {'query_id': ['qb_report','run_id']}